In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import nltk
from torchtext.data import Field, Iterator
from konlpy.tag import Kkma
kor_tagger = Kkma()

In [2]:
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Word similarity

In [3]:
# 단어 간의 유사도 판정
# 단어를 임의의 tensor에 매칭
pytorch = torch.Tensor([0.6,-0.2,0.7,0.3,0.7,-0.2,0.1,0.1]).to(device)
tensorflow = torch.Tensor([0.4,-0.1,0.6,-0.2,0.6,-0.2,0.3,0.4]).to(device)
cat = torch.Tensor([-0.3,0.2,0.1,0.2,-0.2,0.1,-0.3,0.1]).to(device)

In [10]:
# 유사도 판정 방법 [1]
# 단어 간의 inner product 수행
pytorch_dot_cat = pytorch.dot(cat)
print(pytorch_dot_cat.item())

-0.26999998092651367


In [11]:
pytorch_dot_tensorflow = pytorch.dot(tensorflow)
print(pytorch_dot_tensorflow.item())

1.1500000953674316


In [12]:
# 유사도 판정 방법 [2]
# cosine_similarity 함수 이용
sim1 = F.cosine_similarity(pytorch.view(1, -1), tensorflow.view(1, -1))
print("pytorch-tensorflow similarity:", sim1.item())

pytorch-tensorflow similarity: 0.8417288661003113


In [13]:
sim2 = F.cosine_similarity(pytorch.view(1, -1), cat.view(1, -1))
print("pytorch-cat similarity:", sim2.item())

pytorch-cat similarity: -0.37998029589653015


# Word Embedding

In [19]:
num_words = 7 # 총 단어의 갯수
output_dims = 5 # embedding 시킬 차원
embed = nn.Embedding(num_words, output_dims)

print(embed.weight)
print(embed.weight.shape)

Parameter containing:
tensor([[ 1.2731,  1.0128,  0.0141, -0.2883,  0.3565],
        [ 1.5651, -0.8678, -0.9753,  0.5070,  1.1181],
        [ 1.3116, -1.0805,  0.2380,  0.6695,  0.3273],
        [ 0.8691,  0.8039,  1.7240,  0.3429, -0.7110],
        [-0.8318,  0.0295, -0.7858, -1.9219, -0.1093],
        [-0.4683, -0.7369, -0.5337,  0.0878, -0.7929],
        [-0.8846, -0.7385, -0.9770, -1.3635,  0.1110]])
torch.Size([7, 5])


In [20]:
apple = torch.LongTensor([2])
print(apple)
apple_emb = embed(apple)
print(apple_emb)

tensor([ 2])
tensor([[ 1.3116, -1.0805,  0.2380,  0.6695,  0.3273]])


In [21]:
sentence = torch.LongTensor([1, 3, 5]) # 단어의 sequence (문장)
print(sentence)
sentence_emb = embed(sentence)
print(sentence_emb)

tensor([ 1,  3,  5])
tensor([[ 1.5651, -0.8678, -0.9753,  0.5070,  1.1181],
        [ 0.8691,  0.8039,  1.7240,  0.3429, -0.7110],
        [-0.4683, -0.7369, -0.5337,  0.0878, -0.7929]])


# Sentence embedding
각 word vector를 합하거나 평균을 구해서 하나의 vector를 반환.
=> 문장 단위로 embedding

In [25]:
# EmbeddingBag의 mode에는 'mean'과 'sum'이 올 수 있음.
emb_sum = nn.EmbeddingBag(num_words, output_dims, mode='mean')
sentence = torch.LongTensor([[1, 3, 5]]).to(device)
sentence_emb = emb_sum(sentence)

print("input sentence:\n", sentence)
print("\n")
print("embedding output sentence:\n", sentence_emb)

input sentence:
 tensor([[ 1,  3,  5]])


embedding output sentence:
 tensor([[ 0.3276, -0.1942, -1.0651, -0.0037,  0.8308]])


In [26]:
corpus = ["안녕하세요 저는 파이토치를 공부중입니다.","파이토치는 딥러닝 라이브러리이다", "파이토치와 유사한 것으로 텐서플로우와 케라스 등이 있다",
         "파이토치는 정말 쉽다","텐서플로우는 구글이 만들었다","페북에서는 파이토치를 만들었다", "파이썬과 쉽게 사용할 수 있는 것이 장점이다",
         "그 중 특히 자연어처리할 때 파이토치가 좋다","하지만 아직 베타 버전인게 파이토치의 단점이다","원래는 루아라는 언어로 만들어진 토치라는 프레임워크였다",
         "텐서플로우나 파이토치는 자동미분 기능을 제공한다", "이를 이용하면 딥러닝 모델을 쉽게 만들 수 있다"]

In [30]:
# 먼저 corpus의 문장들을 tokenize
tokenized = [kor_tagger.morphs(sentence) for sentence in corpus]
print(tokenized)

[['안녕', '하', '세요', '저', '는', '파이', '토치', '를', '공부', '중', '이', 'ㅂ니다', '.'], ['파이', '토치', '는', '딥', '러닝', '라이브러리', '이', '다'], ['파이', '토치', '와', '유사', '하', 'ㄴ', '것', '으로', '텐서플로우', '와', '하', '게', '라스', '등', '이', '있', '다'], ['파이', '토치', '는', '정말', '쉽', '다'], ['텐서플로우', '는', '구', '글', '이', '만들', '었', '다'], ['페북에서', '는', '파이', '토치', '를', '만들', '었', '다'], ['파이', '썰', 'ㄴ', '과', '쉽', '게', '사용', '하', 'ㄹ', '수', '있', '는', '것', '이', '장점', '이', '다'], ['그', '중', '특히', '자연어', '처리', '하', 'ㄹ', '때', '파이', '토치', '가', '좋', '다'], ['하지만', '아직', '베타', '버전', '이', 'ㄴ', '것', '이', '파이', '토치', '의', '단점', '이', '다'], ['원래', '는', '루', '아', '이', '라는', '언어', '로', '만들', '어', '지', 'ㄴ', '토치', '이', '라는', '프레임', '워크', '이', '었', '다'], ['텐서플로우', '나', '파이', '토치', '는', '자동', '미분', '기능', '을', '제공', '하', 'ㄴ다'], ['이르', 'ㄹ', '이용', '하', '면', '딥', '러닝', '모델', '을', '쉽', '게', '만들', 'ㄹ', '수', '있', '다']]


In [33]:
# tokenize된 문장들을 대상으로 word2idx 세팅
word2idx = {}
for sentence in tokenized:
    for token in sentence:
        if word2idx.get(token) == None:
            word2idx[token] = len(word2idx)
print(word2idx)
print(len(word2idx))

{'안녕': 0, '하': 1, '세요': 2, '저': 3, '는': 4, '파이': 5, '토치': 6, '를': 7, '공부': 8, '중': 9, '이': 10, 'ㅂ니다': 11, '.': 12, '딥': 13, '러닝': 14, '라이브러리': 15, '다': 16, '와': 17, '유사': 18, 'ㄴ': 19, '것': 20, '으로': 21, '텐서플로우': 22, '게': 23, '라스': 24, '등': 25, '있': 26, '정말': 27, '쉽': 28, '구': 29, '글': 30, '만들': 31, '었': 32, '페북에서': 33, '썰': 34, '과': 35, '사용': 36, 'ㄹ': 37, '수': 38, '장점': 39, '그': 40, '특히': 41, '자연어': 42, '처리': 43, '때': 44, '가': 45, '좋': 46, '하지만': 47, '아직': 48, '베타': 49, '버전': 50, '의': 51, '단점': 52, '원래': 53, '루': 54, '아': 55, '라는': 56, '언어': 57, '로': 58, '어': 59, '지': 60, '프레임': 61, '워크': 62, '나': 63, '자동': 64, '미분': 65, '기능': 66, '을': 67, '제공': 68, 'ㄴ다': 69, '이르': 70, '이용': 71, '면': 72, '모델': 73}
74


In [34]:
# embedding matrix 생성
# (단어의 총 갯수, 임베딩 결과의 차원)
emb_mat = nn.Embedding(len(word2idx), 10)
print(emb_mat.weight)
print(emb_mat.weight.shape)

Parameter containing:
tensor([[ 0.9451,  0.5750, -0.0413, -0.1526, -0.7329, -0.9606,  1.2344,
          1.6398, -1.3034,  0.5405],
        [-1.4281,  0.5190,  0.2732,  0.9950,  0.2831,  0.0026, -0.7226,
         -1.4783, -0.3876,  0.8144],
        [ 1.9244, -0.8357,  0.5795, -0.3599,  1.2293,  1.5004,  0.0789,
          0.0448,  0.4895,  1.4768],
        [ 2.8679,  0.2165,  0.1807, -1.5286, -0.7992, -0.8839,  0.1912,
         -0.3952, -0.3263,  0.1190],
        [ 0.0678, -0.2030, -1.3602, -1.7874, -0.9465, -1.2570, -0.4823,
          1.0512,  0.0312, -0.2953],
        [-0.6527, -0.7597,  0.0742, -0.0277,  0.0450,  0.2152, -0.6927,
          0.7325, -0.1009, -1.2223],
        [-0.3869,  0.8482, -0.0701, -1.0024,  1.2641,  0.8975, -0.6477,
          0.3428,  0.6561,  0.8522],
        [-0.0065, -0.0353,  0.1221,  0.4386, -1.0480, -1.5049,  1.2756,
          0.2893, -0.6522, -1.9495],
        [-0.1191,  0.1869,  1.8410, -0.0215,  0.3923, -0.3457, -1.0933,
         -0.7827, -0.4927,  1.1694

In [35]:
print(word2idx["토치"])

6


In [36]:
pytorch_tensor = torch.LongTensor([word2idx["토치"]])
tensorflow_tensor = torch.LongTensor([word2idx["텐서플로우"]])
print(pytorch_tensor)
print(tensorflow_tensor)

tensor([ 6])
tensor([ 22])


In [37]:
pytorch_vector = emb_mat(pytorch_tensor)
tensorflow_vector = emb_mat(tensorflow_tensor)
print(pytorch_vector)
print(tensorflow_vector)

tensor([[-0.3869,  0.8482, -0.0701, -1.0024,  1.2641,  0.8975, -0.6477,
          0.3428,  0.6561,  0.8522]])
tensor([[-0.7032, -0.1315,  0.2338,  1.2068, -0.7824,  0.3204,  2.0527,
          0.0087, -0.6999,  1.9060]])


In [38]:
print(pytorch_vector.shape)
print(tensorflow_vector.shape)

torch.Size([1, 10])
torch.Size([1, 10])


In [40]:
pytorch_vector = pytorch_vector.squeeze()
tensorflow_vector = tensorflow_vector.squeeze()

In [42]:
print(pytorch_vector.shape)
print(tensorflow_vector.shape)

torch.Size([10])
torch.Size([10])


In [41]:
# 이제 "토치"와 "텐서플로우"의 유사도 계산
pytorch_vector.dot(tensorflow_vector)

tensor(-1.9284)